In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open('names-shuffled.txt', 'r').read().splitlines()

In [3]:
chars = sorted(list(set(''.join(words))))
stoi = {c:i+1 for i, c in enumerate(chars)}
stoi['.'] = 0
itos = {v:k for k, v in stoi.items()}
vocab_size = len(itos)
print(vocab_size)

27


In [4]:
context_length = 3 # context length: number of chars used to predict the next char

def build_dataset(words):
    X, Y = [], [] # inputs and labels

    for w in words:
        context = [0] * context_length
        for c in w + '.':
            X.append(context)
            Y.append(stoi[c])
            context = context[1:] + [stoi[c]] # sliding the context by 1 character to right

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random

random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])      # 80%
Xdev, Ydev = build_dataset(words[n1:n2])  # 10%
Xte, Yte = build_dataset(words[n2:])      # 10%

torch.Size([404420, 3]) torch.Size([404420])
torch.Size([50599, 3]) torch.Size([50599])
torch.Size([50438, 3]) torch.Size([50438])


In [5]:
# to compare manual gradients with PyTorch gradients

def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item() # exactly equal
    app = torch.allclose(dt, t.grad)    # approx equal
    maxdiff = (dt - t.grad).abs().max().item() # maximum difference
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [6]:
n_emb_dim = 10    # embedding dimensionality
n_hidden = 64    # neurons in hidden layers

g = torch.Generator().manual_seed(1337101)

C = torch.randn((vocab_size, n_emb_dim), generator=g)
# Layer 1
W1 = torch.randn((context_length*n_emb_dim, n_hidden), generator=g) * (5/3)/((context_length*n_emb_dim)**0.5)
b1 = torch.randn(n_hidden, generator=g) * 0.1 # useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1

# batch normalization params
bn_gain = torch.ones((1, n_hidden))*0.1 + 1.0
bn_bias = torch.zeros((1, n_hidden))*0.1

# grads not required for these
# bn_mean_running = torch.zeros((1, n_hidden))
# bn_std_running = torch.ones((1, n_hidden))

# added bn_gain and bn_bias
parameters = [C, W1, b1, W2, b2, bn_gain, bn_bias]

for p in parameters:
    p.requires_grad = True

print(f'Total Parameters: {sum(p.nelement() for p in parameters)}')

Total Parameters: 4137


In [7]:
batch_size = 24
n = batch_size # variable for convenience

# creating a mini-batch
ix = torch.randint(0, Xtr.shape[0], (batch_size, ), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

In [33]:
# forward pass

emb = C[Xb]
embcat = emb.view(-1, W1.shape[0])
# Linear layer 1
hprebn = embcat@W1 + b1 # hidden layer pre-BN
# BN layer
bnmeani = (1/n)*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = (1/(n-1))*(bndiff2).sum(0, keepdim=True) # Bessel's Correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**(-0.5) # 1/sqrt(bnvar + 1e-5)
bnraw = bndiff*bnvar_inv
hpreact = bn_gain * bnraw + bn_bias # hidden layer pre-activation
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h@W2 + b2 # output layer
# cross entropy loss (F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numeric stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdim=True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean() # indexing the correct (Yth) char prob from each 24 (n) rows

# PyTorch backward pass
for p in parameters:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv,
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
          bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
          embcat, emb]:
    t.retain_grad()
loss.backward()
loss


tensor(3.4895, grad_fn=<NegBackward0>)

In [99]:
# Exercise 1: backprop through the whole thing manually, 
# backpropagating through exactly all of the variables 
# as they are defined in the forward pass above, one by one

# -----------------
# YOUR CODE HERE :)

dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1/n

dprobs = dlogprobs * (1.0/probs)

dcounts_sum_inv = (dprobs * counts).sum(1, keepdim=True) # when a node is replicated then its gradient is added during backpropagation

dcounts_sum = dcounts_sum_inv * (-counts_sum**-2)

dcounts = (dprobs * counts_sum_inv) + (dcounts_sum * torch.ones_like(counts))

# -----------------

cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
# cmp('norm_logits', dnorm_logits, norm_logits)
# cmp('logit_maxes', dlogit_maxes, logit_maxes)
# cmp('logits', dlogits, logits)
# cmp('h', dh, h)
# cmp('W2', dW2, W2)
# cmp('b2', db2, b2)
# cmp('hpreact', dhpreact, hpreact)
# cmp('bngain', dbngain, bngain)
# cmp('bnbias', dbnbias, bnbias)
# cmp('bnraw', dbnraw, bnraw)
# cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
# cmp('bnvar', dbnvar, bnvar)
# cmp('bndiff2', dbndiff2, bndiff2)
# cmp('bndiff', dbndiff, bndiff)
# cmp('bnmeani', dbnmeani, bnmeani)
# cmp('hprebn', dhprebn, hprebn)
# cmp('embcat', dembcat, embcat)
# cmp('W1', dW1, W1)
# cmp('b1', db1, b1)
# cmp('emb', demb, emb)
# cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: False | approximate: True  | maxdiff: 2.384185791015625e-07
counts_sum_inv  | exact: False | approximate: True  | maxdiff: 5.960464477539063e-08
counts_sum      | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10
counts          | exact: False | approximate: True  | maxdiff: 2.9802322387695312e-08
